In [6]:
!pip install keras

  Using cached https://files.pythonhosted.org/packages/44/e1/dc0757b20b56c980b5553c1b5c4c32d378c7055ab7bfa92006801ad359ab/Keras-2.4.3-py2.py3-none-any.whl


You are using pip version 10.0.1, however version 20.3.4 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [191]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [192]:
data = pd.read_csv('data_2021_spring/train.csv')
# Keeping only the neccessary columns
data = data[['text','stars']]

In [217]:
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X, maxlen = 800)

In [207]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(5,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 800, 128)          256000    
_________________________________________________________________
spatial_dropout1d_4 (Spatial (None, 800, 128)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 985       
Total params: 511,785
Trainable params: 511,785
Non-trainable params: 0
_________________________________________________________________
None


In [196]:
Y = pd.get_dummies(data['stars']).values
print(X.shape,Y.shape)

(10000, 800) (10000, 5)


In [197]:
data2 = pd.read_csv('data_2021_spring/valid.csv')
# Keeping only the neccessary columns
data2 = data2[['text','stars']]

In [198]:
data2.head()

,text,stars
0,Small coffee shop on the corner. Good coffee. ...,5
1,Food was okay. Eating off styrofoam with plast...,3
2,Not at all impressed with Jet...maybe its beca...,2
3,"We got a 14"" pepperoni pizza I'd say it feeds ...",4
4,Unfortunately this location has terrible custo...,1


In [218]:
data2['text'] = data2['text'].apply(lambda x: x.lower())
data2['text'] = data2['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
    
max_fatures = 2000
# tokenizer.fit_on_texts(data2['text'].values)
X_valid = tokenizer.texts_to_sequences(data2['text'].values)
X_valid = pad_sequences(X_valid, maxlen = 800)

In [219]:
Y_train = pd.get_dummies(data['stars']).values
Y_valid = pd.get_dummies(data2['stars']).values

In [201]:
X_valid

array([[   0,    0,    0, ...,  260,  152,  114],
       [   0,    0,    0, ...,  420,  587,  145],
       [   0,    0,    0, ...,   18,   62,    5],
       ...,
       [   0,    0,    0, ..., 1254,  104,  500],
       [   0,    0,    0, ...,  197, 1803, 1417],
       [   0,    0,    0, ...,    2,   32,   31]])

In [202]:
Y_valid.shape

(2000, 5)

In [204]:
X_train = X
print(X_train.shape,Y_train.shape)
print(X_valid.shape,Y_valid.shape)

(10000, 800) (10000, 5)
(2000, 800) (2000, 5)


In [208]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 12, batch_size=batch_size, verbose = 1, validation_data = (X_valid, Y_valid))

Epoch 1/12
313/313 [==============================] - 888s 3s/step - accuracy: 0.4141 - loss: 1.3434 - val_accuracy: 0.2505 - val_loss: 1.7864
Epoch 2/12
313/313 [==============================] - 865s 3s/step - accuracy: 0.5516 - loss: 1.0628 - val_accuracy: 0.2455 - val_loss: 2.0277
Epoch 3/12
313/313 [==============================] - 879s 3s/step - accuracy: 0.5920 - loss: 0.9674 - val_accuracy: 0.2360 - val_loss: 2.0545
Epoch 4/12
313/313 [==============================] - 870s 3s/step - accuracy: 0.6191 - loss: 0.9172 - val_accuracy: 0.2245 - val_loss: 2.2631
Epoch 5/12
313/313 [==============================] - 866s 3s/step - accuracy: 0.6453 - loss: 0.8533 - val_accuracy: 0.2305 - val_loss: 2.2140
Epoch 6/12
313/313 [==============================] - 880s 3s/step - accuracy: 0.6701 - loss: 0.8181 - val_accuracy: 0.2165 - val_loss: 2.4112
Epoch 7/12
313/313 [==============================] - 868s 3s/step - accuracy: 0.6943 - loss: 0.7537 - val_accuracy: 0.2370 - val_loss: 2.5288

In [220]:
score,acc = model.evaluate(X_valid, Y_valid, verbose = 1, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

63/63 [==============================] - 8s 129ms/step - accuracy: 0.5565 - loss: 1.2793
score: 1.28
acc: 0.56


In [164]:
data3 = pd.read_csv('data_2021_spring/test.csv')
# Keeping only the neccessary columns
data3 = data3[['text', 'useful']]

In [165]:
data3['text'] = data3['text'].apply(lambda x: x.lower())
data3['text'] = data3['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

for idx,row in data3.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer3 = Tokenizer(num_words=max_fatures, split=' ')
tokenizer3.fit_on_texts(data3['text'].values)
X_test = tokenizer2.texts_to_sequences(data3['text'].values)
X_test = pad_sequences(X_test)

In [166]:
results = model.predict(X_test, batch_size = batch_size, verbose = 1)
print(results)

63/63 [==============================] - 8s 130ms/step
[[2.78522447e-02 9.11187291e-01 6.06728978e-02 2.56118219e-04
  3.14944991e-05]
 [3.10580246e-03 3.56117473e-03 3.74854282e-02 7.94116616e-01
  1.61731049e-01]
 [3.47217292e-01 4.83162522e-01 1.64265707e-01 1.01487339e-03
  4.33964469e-03]
 ...
 [1.35483255e-03 2.67890375e-03 3.27204727e-02 5.53886473e-01
  4.09359366e-01]
 [7.24423766e-01 2.15039581e-01 3.36555019e-02 6.83586393e-03
  2.00452413e-02]
 [2.07964284e-03 1.70034748e-02 4.69133914e-01 4.06690061e-01
  1.05092935e-01]]


In [167]:
results

array([[2.78522447e-02, 9.11187291e-01, 6.06728978e-02, 2.56118219e-04,
        3.14944991e-05],
       [3.10580246e-03, 3.56117473e-03, 3.74854282e-02, 7.94116616e-01,
        1.61731049e-01],
       [3.47217292e-01, 4.83162522e-01, 1.64265707e-01, 1.01487339e-03,
        4.33964469e-03],
       ...,
       [1.35483255e-03, 2.67890375e-03, 3.27204727e-02, 5.53886473e-01,
        4.09359366e-01],
       [7.24423766e-01, 2.15039581e-01, 3.36555019e-02, 6.83586393e-03,
        2.00452413e-02],
       [2.07964284e-03, 1.70034748e-02, 4.69133914e-01, 4.06690061e-01,
        1.05092935e-01]], dtype=float32)

In [168]:
df = pd.DataFrame(results, columns = ['1', '2', '3', '4', '5'])
df

,1,2,3,4,5
0,0.027852,0.911187,0.060673,0.000256,0.000031
1,0.003106,0.003561,0.037485,0.794117,0.161731
2,0.347217,0.483163,0.164266,0.001015,0.004340
3,0.042479,0.327837,0.333459,0.272492,0.023733
4,0.019480,0.135372,0.509945,0.272626,0.062577
...,...,...,...,...,...
1995,0.010049,0.654123,0.325179,0.009888,0.000761
1996,0.010083,0.095824,0.363412,0.437269,0.093411
1997,0.001355,0.002679,0.032720,0.553886,0.409359
1998,0.724424,0.215040,0.033656,0.006836,0.020045


In [81]:
data2

,text,stars
0,nice to have a diner still around food was goo...,4
1,tried this a while back got the fried chicken ...,2
2,i expected more pork selections on menu food v...,4
3,yummy this place is phenomenal it is pricey bu...,4
4,the truffle macaroni cheese and potatoes au g...,5
...,...,...
9995,i think most people like this place because of...,2
9996,thought id give this location a try from the o...,1
9997,i first tried this place a while ago and ever ...,4
9998,yummy yummy yummy i got the chicken fried stea...,5


In [221]:
model.save("second_attempt")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: second_attempt\assets
